In [16]:
from sklearn import preprocessing as pp
import numpy as np
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
from keras.utils import np_utils

def buildNotes():
    notes = []
    for file in glob.glob("Music/*.mid"):
        try:
            midi = converter.parse(file)              
        except:
            print("MIDI file %s failed to parse" % file)
            continue
        print("Parsing %s" % file)
        notes_to_parse = None
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    pickle.dump(notes, open('notes.p', 'wb'))

    return notes

def prepareSeq(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 4 

    pitchnames = sorted(set(item for item in notes))
#    print(pitchnames)

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
#    print(note_to_int)

    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

notes = buildNotes()
netIn, netOut = prepareSeq(notes, len(notes))

print(netIn)
print(netOut)

Parsing Music\Fugue1.mid
Parsing Music\Fugue10.mid
Parsing Music\Fugue11.mid
MIDI file Music\Fugue12.mid failed to parse
Parsing Music\Fugue13.mid
Parsing Music\Fugue14.mid
Parsing Music\Fugue15.mid
Parsing Music\Fugue16.mid
Parsing Music\Fugue17.mid
Parsing Music\Fugue18.mid
MIDI file Music\Fugue19.mid failed to parse
Parsing Music\Fugue2.mid
Parsing Music\Fugue20.mid
Parsing Music\Fugue21.mid
Parsing Music\Fugue22.mid
Parsing Music\Fugue23.mid
Parsing Music\Fugue24.mid
Parsing Music\Fugue3.mid
Parsing Music\Fugue4.mid
Parsing Music\Fugue5.mid
MIDI file Music\Fugue6.mid failed to parse
Parsing Music\Fugue7.mid
Parsing Music\Fugue8.mid
Parsing Music\Fugue9.mid
[[[0.00214496]
  [0.0023684 ]
  [0.00272589]
  [0.00308339]]

 [[0.0023684 ]
  [0.00272589]
  [0.00308339]
  [0.00344088]]

 [[0.00272589]
  [0.00308339]
  [0.00344088]
  [0.00308339]]

 ...

 [[0.0020109 ]
  [0.00178747]
  [0.0016981 ]
  [0.00142998]]

 [[0.00178747]
  [0.0016981 ]
  [0.00142998]
  [0.00277058]]

 [[0.0016981 ]
